# TSP - Traveling Salesman Problem

In [163]:
import math
import copy
import time
import numpy as np

## Auxiliary methods

In [164]:
def import_dataset(path: str):
    """
    :param path: path to the dataset
    :return: numbers of vertexes, weight type, list of vertexes with tag and coordinates
    """
    lines = open(path, "r").read().split("\n")

    cont = 0
    dim, wt, v = 0, "", []
    while not lines[cont].startswith("NODE_COORD_SECTION"):
        if lines[cont].startswith("DIMENSION"):
            dim = int(lines[cont].split(":")[1][1:])
        elif lines[cont].startswith("EDGE_WEIGHT_TYPE"):
            wt = lines[cont].split(":")[1][1:]
        cont += 1
    cont += 1

    for i in range(cont, len(lines)):
        line = lines[i].split()
        if len(line) < 3: # Skip EOF
            break
        tag, x, y = int(line[0]) - 1, float(line[1]), float(line[2])
        v.append((tag, [x, y])) #(i, [x_value, y_value])

    return dim, wt, v

In [165]:
def weight (u, v):
    """
    :param u: vertex u
    :param v: vertex v
    :return: euclidean or geographical distance between u and v
    """
    if weight_type == "EUC_2D": # Euclidean distance
        return round(math.sqrt(sum([(a - b) ** 2 for a, b in zip(u, v)])))
    else: # Geographical distance
        PI = 3.141592
        deg_xu = int(u[0])
        min_xu = u[0] - deg_xu
        rad_xu = PI * (deg_xu + 5.0 * min_xu/ 3.0) / 180.0

        deg_yu = int(u[1])
        min_yu = u[1] - deg_yu
        rad_yu = PI * (deg_yu + 5.0 * min_yu/ 3.0) / 180.0

        deg_xv = int(v[0])
        min_xv = v[0] - deg_xv
        rad_xv= PI * (deg_xv + 5.0 * min_xv/ 3.0) / 180.0

        deg_yv = int(v[1])
        min_yv = v[1]- deg_yv
        rad_yv = PI * (deg_yv + 5.0 * min_yv/ 3.0) / 180.0

        RRR = 6378.388
        q1 = math.cos(rad_yu - rad_yv)
        q2 = math.cos(rad_xu - rad_xv)
        q3 = math.cos(rad_xu + rad_xv)
        return int(RRR * math.acos(0.5 * ((1.0 + q1) * q2 - (1.0 - q1) * q3)) + 1.0)

## Held and Karp algorithm

In [166]:
# Build a unique identifier for the subset S, joining the indexes of the vertexes with a blank space between them
def encode(S):
  encoded_string = ""
  for s in S:
    encoded_string += " " + str(s[0])
  return encoded_string

In [167]:
start = time.time()
def held_karp (v, S): # v: arrival vertex of S starting from 0, S: subset of vertexes
  S_index = subsets[encode(S)] # Build a unique identifier for the subset S
  if (time.time() - start) > 180: # Max time: 3 min
    return None
  elif (len(S) == 1) & (S[0][0] == v): # Base case: the solution is the weight of the edge {v, 0}
    return weight(vertexes[v][1], vertexes[0][1])
  elif d[v, S_index] != 0: # Distance already computed
    return d[v, S_index]
  else:  # Recursive case: find the minimum among all the sub-paths
    mindist = math.inf
    minprec = None
    subset = [i for i in S if i[0] != v] # S \ {v}
    if encode(subset) not in subsets:
      global counter
      subsets[encode(subset)] = counter
      counter += 1
    for u in subset:
      dist = held_karp(u[0], subset) # Compute the partial result
      if dist is None:
        break
      else:
        w = weight(u[1], vertexes[v][1])
        if (dist + w) < mindist:
          mindist = dist + w
          minprec = u[0]
    d[v, S_index] = mindist # Update d with the minimum distance
    phi[v, S_index] = minprec # Update phi with predecessor of v
    return mindist

## Constructive heuristic algorithm with nearest neighbor

In [168]:
def closest_vertex(p, V: list):
    """
    :param p: last vertex of the partial path
    :param V: list of vertexes not added yet
    :return: the closest vertex to p and the distance between them
    """
    min_dist = math.inf
    closest = None
    for v in V:
        dist = weight(p[1], v[1])
        if dist < min_dist:
            min_dist = dist
            closest = v
    return closest, min_dist

In [169]:
def nearest_neighbor(V: list):
    """
    :param V: list of initial vertexes
    :return: TSP shortest path and its cost using Nearest Neighbor construcitve heuristic
    """
    shortest_distance = math.inf
    for i in range(0, len(V)):
        ''' Initialization '''
        unvisited = copy.deepcopy(V)      # Deep copy of V in order to maintain it unchanged
        starting_vertex = unvisited[i]    # Starting vertex
        partial_path = [starting_vertex]
        unvisited.remove(starting_vertex)
        total_dist = 0
        while len(unvisited) > 0:
            ''' Selection '''
            closest, dist = closest_vertex(partial_path[-1], unvisited)
            ''' Insertion '''
            partial_path.append(closest)
            unvisited.remove(closest)
            total_dist += dist
        total_dist += weight(partial_path[-1][1], starting_vertex[1])
        partial_path.append(starting_vertex) # Add the starting vertex to close the cycle
        if total_dist < shortest_distance: # Saving the shortest path and its cost
            shortest_distance = total_dist
    return shortest_distance

## 2-approx algorithm

## Main

In [170]:
def execute_algs(dataset_name):
    print("-------------------------------------------------------------------")
    print("Executing Held Karp algorithm on {} dataset.".format(dataset_name))
    starting_time = time.time()
    held_karp_value = held_karp(0, vertexes)
    held_karp_time = '%.5f'%(time.time() - starting_time)
    print("Best solution found in {} seconds : {}".format(held_karp_time, held_karp_value))

    print("Executing Nearest Neighbor algorithm on {} dataset.".format(dataset_name))
    starting_time = time.time()
    nearest_neighbor_value = nearest_neighbor(vertexes)
    nearest_neighbor_time = '%.5f'%(time.time() - starting_time)
    print("Best solution found in {} seconds : {}".format(nearest_neighbor_time, nearest_neighbor_value))

    #print("Executing 2-Approximation algorithm on {} dataset.".format(dataset_name))
    #starting_time = time.time()
    #two_approximation_value = two_approximation()
    #two_approximation_time = '%.5f'%(time.time() - starting_time)
    #print("Best solution found in {} seconds : {}".format(two_approximation_time, two_approximation_value))
    #print()

    held_karp_error = '%.2f'%(((held_karp_value - inputs[dataset_name]) / inputs[dataset_name])*100)
    nearest_neighbor_error = '%.2f'%(((nearest_neighbor_value - inputs[dataset_name]) / inputs[dataset_name])*100)
    #two_approximation_error = '%.2f'%(((two_approximation_value- optimal_solutions[dataset_name]) / optimal_solutions[dataset_name])*100)

    return [
            held_karp_value, held_karp_time, held_karp_error,
            nearest_neighbor_value, nearest_neighbor_time, nearest_neighbor_error
            #two_approximation_value, two_approximation_time, two_approximation_error
    ]

In [171]:
path= "tsp_dataset/"
results = []
inputs = {
    "burma14.tsp": 3323
    #"ulysses16.tsp": 6859,
    #"ulysses22.tsp": 7013,
    #"eil51.tsp": 426,
    #"berlin52.tsp": 7542,
    #"kroD100.tsp": 21294,
    #"kroA100.tsp": 21282,
    #"ch150.tsp": 6528,
    #"gr202.tsp": 40160,
    #"gr229.tsp": 134602,
    #"pcb442.tsp": 50778 ,
    #"d493.tsp":	35002,
    #"dsj1000.tsp":18659688
}
for dataset_name in inputs.keys():
    dimension, weight_type, vertexes = import_dataset(path + dataset_name)
    # Held and Karp initialization section
    subsets = {} # Dictionary to enumerate the subsets
    counter = 0
    subsets[encode(vertexes)] = counter # Add the first subset, with all vertexes
    counter += 1
    d = np.zeros(shape = (dimension, 2 ** (dimension - 1))) # d[v, S]: distance of the TSP starting from 0 to v, passing through al points of S
    phi = np.zeros(shape = (dimension, 2 ** (dimension - 1))) # phi[v, S]: predecessor of v

    results.append(execute_algs(dataset_name))

-------------------------------------------------------------------
Executing Held Karp algorithm on burma14.tsp dataset.
Best solution found in 2.42518 seconds : 3323.0
Executing Nearest Neighbor algorithm on burma14.tsp dataset.
Best solution found in 0.00415 seconds : 3841


## Results

## Plotting the results